In [88]:
! curl "http://localhost:8000"

Not found
This is a 4store SPARQL server 
Check /status/ for more info


In [89]:
import vcf

# REPLACE pjotr.me with your name!!

In [90]:
print("@prefix : <http://jane.me/rdf/ns#> .\n")

vcf_reader = vcf.Reader(open('/home/jupyter/jupyter/contrib/gatk_exome.vcf', 'r'))
count=0
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    print(":"+name)
    print("        :query_id \""+name+"\" ;")
    print("        :chr \""+record.CHROM+"\" ;")
    print("        :pos "+str(record.POS)+" ;")
    print("        :alt \""+str(record.ALT[0])+"\" .")
    count +=1
    if count > 5:
        break


@prefix : <http://jane.me/rdf/ns#> .

:chr1_22848972_C
        :query_id "chr1_22848972_C" ;
        :chr "1" ;
        :pos 22848972 ;
        :alt "C" .
:chr1_33133968_C
        :query_id "chr1_33133968_C" ;
        :chr "1" ;
        :pos 33133968 ;
        :alt "C" .
:chr1_60466814_C
        :query_id "chr1_60466814_C" ;
        :chr "1" ;
        :pos 60466814 ;
        :alt "C" .
:chr1_60466840_T
        :query_id "chr1_60466840_T" ;
        :chr "1" ;
        :pos 60466840 ;
        :alt "T" .
:chr1_62516683_C
        :query_id "chr1_62516683_C" ;
        :chr "1" ;
        :pos 62516683 ;
        :alt "C" .
:chr1_66058513_G
        :query_id "chr1_66058513_G" ;
        :chr "1" ;
        :pos 66058513 ;
        :alt "G" .


### Now write the same to file with samples and a few extra triples

In [91]:

import vcf

buf = """
@prefix    :   <http://jane.me/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

"""

vcf_reader = vcf.Reader(open('/home/jupyter/jupyter/contrib/gatk_exome.vcf', 'r'))
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    buf += ":"+name+"\n"
    buf += "  :variant \""+name+"\" ;\n"
    buf += "  rdf:type :snp ;\n"
    buf += "  :chr \""+record.CHROM+"\" ;\n"
    buf += "  :pos \""+str(record.POS)+"\" ;\n"
    buf += "  :alt \""+str(record.ALT[0])+"\" .\n"
    for sample in record.samples:
        buf += ":"+name+"_"+str(sample.sample)+"\n"
        if sample["DP"]:
            buf += "  :dp   "+str(sample["DP"])+" ;\n"
        buf += "  :vcf :"+name+" .\n"

# Finally we connect a few SNPs

buf += """
:chr1_66058513_G  :controls  :chr1_114354942_G .
:chr1_114354942_G :controls  :chr1_144873962_C .
:chr1_144873962_C :controls  :chr1_66058513_G .
:chr1_144873962_C :controls  :chrX_134713855_A .
"""

# Write to file

print(buf)
f = open("test.rdf","w") # note this file is shared!
f.write(buf)




@prefix    :   <http://jane.me/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

:chr1_22848972_C
  :variant "chr1_22848972_C" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "22848972" ;
  :alt "C" .
:chr1_22848972_C_CONTROLP25
  :dp   2 ;
  :vcf :chr1_22848972_C .
:chr1_22848972_C_CONTROLP26
  :vcf :chr1_22848972_C .
:chr1_33133968_C
  :variant "chr1_33133968_C" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "33133968" ;
  :alt "C" .
:chr1_33133968_C_CONTROLP25
  :vcf :chr1_33133968_C .
:chr1_33133968_C_CONTROLP26
  :dp   2 ;
  :vcf :chr1_33133968_C .
:chr1_60466814_C
  :variant "chr1_60466814_C" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "60466814" ;
  :alt "C" .
:chr1_60466814_C_CONTROLP25
  :vcf :chr1_60466814_C .
:chr1_60466814_C_CONTROLP26
  :dp   2 ;
  :vcf :chr1_60466814_C .
:chr1_60466840_T
  :variant "chr1_60466840_T" ;
  rdf:type :snp ;
  :chr "1" ;
  :pos "60466840" ;
  :alt "T" .
:chr1_60466840_T_CONTROLP25
  :vcf :chr1_60466840_T .
:chr1_60466840_T_CONTROLP26
  :

42100

### Validate the file - we can do this in python

In [92]:
# pip3 install --user rdflib
# python3 rdfvalidate.py

import rdflib
g = rdflib.Graph()

result = g.parse('test.rdf', format='n3')
print(len(g))
for stmt in g:
    print(stmt)

# You should see something here...

1418
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr15_65903398_A'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://jane.me/rdf/ns#snp'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr7_100551842_T'), rdflib.term.URIRef('http://jane.me/rdf/ns#pos'), rdflib.term.Literal('100551842'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr3_121168152_T'), rdflib.term.URIRef('http://jane.me/rdf/ns#variant'), rdflib.term.Literal('chr3_121168152_T'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr1_228465370_G'), rdflib.term.URIRef('http://jane.me/rdf/ns#chr'), rdflib.term.Literal('1'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr7_142460335_G'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://jane.me/rdf/ns#snp'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr12_82792942_G'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://jane.me/rdf/ns#snp'))
(rdfl

(rdflib.term.URIRef('http://jane.me/rdf/ns#chr11_124266912_G_CONTROLP26'), rdflib.term.URIRef('http://jane.me/rdf/ns#dp'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr11_63883985_C_CONTROLP26'), rdflib.term.URIRef('http://jane.me/rdf/ns#dp'), rdflib.term.Literal('2', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr22_30771554_G'), rdflib.term.URIRef('http://jane.me/rdf/ns#pos'), rdflib.term.Literal('30771554'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr10_45958881_G_CONTROLP26'), rdflib.term.URIRef('http://jane.me/rdf/ns#vcf'), rdflib.term.URIRef('http://jane.me/rdf/ns#chr10_45958881_G'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#chr1_60466840_T_CONTROLP26'), rdflib.term.URIRef('http://jane.me/rdf/ns#vcf'), rdflib.term.URIRef('http://jane.me/rdf/ns#chr1_60466840_T'))
(rdflib.term.URIRef('http://jane.me/rdf/ns#c

### Here we upload the data to the database

In [93]:
# Equivalent of curl -T $rdf -H 'Content-Type: application/x-turtle' $uri
import requests

myname="jane" # change to your username
rdf="test.rdf" # shared file?!
uri="http://localhost:8000/data/http://biobeat.org/"+myname+"/"+rdf
headers = {'content-type': 'application/x-turtle'}

f = open(rdf)
buf = f.read()
r = requests.put(uri, data=buf, headers=headers)
print(uri)
print(r) # should give a 201


http://localhost:8000/data/http://biobeat.org/jane/test.rdf
<Response [201]>


### Let's check what is in 4store


In [94]:
!lynx --dump localhost:8000/status/size/

                       SPARQL httpd server status - size

KB pythoncourse

   Segment # quads (s) quads (sr) models resources
   0         4162      +0         6      2255
   1         4346      +0         6      2402
     Total   8508      +0         6      4657


### And do a first SPARQL query


In [97]:

end_point = "http://localhost:8000/sparql/"

query = """
PREFIX : <http://jane.me/rdf/jane/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?o WHERE {
    ?s rdf:type ?o .
} LIMIT 10
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?o
<http://mudibo.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>
<http://peter.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>
<http://winfred/rdf/ns#snp>
<http://peter.me/rdf/ns#snp>
<http://jane.me/rdf/ns#snp>
<http://jane.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#snp>
<http://winfred/rdf/ns#snp>



## **Assignment 1**

In [ ]:
#code from uniprot
#Select the UniProt entry with the mnemonic 'A4_HUMAN'

PREFIX up:<http://purl.uniprot.org/core/> 
SELECT ?protein
WHERE
{
        ?protein a up:Protein .
        ?protein up:mnemonic 'A4_HUMAN'
}

In [ ]:
#execution in python

from SPARQLWrapper import SPARQLWrapper, JSON

endpoint = 'https://sparql.uniprot.org/sparql'
sparql = SPARQLWrapper(endpoint)

sparql.setQuery("""
    PREFIX up: <http://purl.uniprot.org/core/>       
    SELECT ?protein
    WHERE 
    { 
            ?protein a up:Protein .
            ?protein up:mnemonic 'A4_HUMAN'
    }
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()
print(results)


# print(results['results']['bindings'])

# import os, ssl
# if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
# getattr(ssl, '_create_unverified_context', None)): 
#     ssl._create_default_https_context = ssl._create_unverified_context

## **Assignment 2**

In [1]:
#2a count the number of identifiers on chr1

import requests 

end_point = "http://localhost:8000/sparql/"

query = """
PREFIX jane: <http://jane.me/rdf/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT (COUNT(?chrom) AS ?chr1_identifiers)

WHERE {
    ?chrom jane:chr \"1\" .
}
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print(r.url)

print(r.text)

?chr1_identifiers
36



In [6]:
#2b assignment is to take that same VCF file and take the cross-referenced identifiers

end_point = "http://localhost:8000/sparql/"

query = """
PREFIX as: <http://jane.me/rdf/ns#>

SELECT ?cros1 ?cros2
WHERE {
    ?cros1 as:controls ?cros2 .
    ?cros1 as:chr "1" .
}

##illegal without grouping?

"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
#print r.url

print(r.text)

?cros1	?cros2
<http://jane.me/rdf/ns#chr1_66058513_G>	<http://jane.me/rdf/ns#chr1_114354942_G>
<http://jane.me/rdf/ns#chr1_66058513_G>	<http://jane.me/rdf/ns#chr1_114354942_G>
<http://jane.me/rdf/ns#chr1_144873962_C>	<http://jane.me/rdf/ns#chrX_134713855_A>
<http://jane.me/rdf/ns#chr1_144873962_C>	<http://jane.me/rdf/ns#chr1_66058513_G>
<http://jane.me/rdf/ns#chr1_144873962_C>	<http://jane.me/rdf/ns#chrX_134713855_A>
<http://jane.me/rdf/ns#chr1_144873962_C>	<http://jane.me/rdf/ns#chr1_66058513_G>
<http://jane.me/rdf/ns#chr1_114354942_G>	<http://jane.me/rdf/ns#chr1_144873962_C>
<http://jane.me/rdf/ns#chr1_114354942_G>	<http://jane.me/rdf/ns#chr1_144873962_C>
<http://jane.me/rdf/ns#chr1_114354942_G>	<http://jane.me/rdf/ns#chr1_144873962_C>
<http://jane.me/rdf/ns#chr1_114354942_G>	<http://jane.me/rdf/ns#chr1_144873962_C>
<http://jane.me/rdf/ns#chr1_66058513_G>	<http://jane.me/rdf/ns#chr1_114354942_G>
<http://jane.me/rdf/ns#chr1_66058513_G>	<http://jane.me/rdf/ns#chr1_114354942_G>
<http://

In [14]:
#2c assignment is to come up with your own unique query. Take a vcf file  and parse it with PyVCF. 
#Generate RDF, so it looks like

# @prefix : <http://biobeat.org/rdf/ns#> .

# :chrX_134713855_A
#   :query_id "chrX_134713855_A";
#   :chr "X" ;
#   :alt "A" ;
#   :pos 134713855 .

# and more of those...


import vcf

jan = """
@prefix : <http://biobeat.org/rdf/ns#> .

"""

vcf_reader = vcf.Reader(open('gatk_exome.vcf', 'r'))
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    jan += ":"+name+"\n"
    jan += "  :query_id \""+name+"\" ;\n"
    jan += "  :chr \""+record.CHROM+"\" ;\n"
    jan += "  :alt \""+str(record.ALT[0])+"\" ;\n"
    jan += "  :pos \""+str(record.POS)+"\" .\n"
    for sample in record.samples:
        jan += ":"+name+"_"+str(sample.sample)+"\n"
        if sample["DP"]:
            jan += "  :dp   "+str(sample["DP"])+" ;\n"
        jan += "  :vcf :"+name+" .\n"

# Write to file

print(jan)
f = open("mytest.rdf","w")
f.write(jan)


@prefix : <http://biobeat.org/rdf/ns#> .

:chr1_22848972_C
  :query_id "chr1_22848972_C" ;
  :chr "1" ;
  :alt "C" ;
  :pos "22848972" .
:chr1_22848972_C_CONTROLP25
  :dp   2 ;
  :vcf :chr1_22848972_C .
:chr1_22848972_C_CONTROLP26
  :vcf :chr1_22848972_C .
:chr1_33133968_C
  :query_id "chr1_33133968_C" ;
  :chr "1" ;
  :alt "C" ;
  :pos "33133968" .
:chr1_33133968_C_CONTROLP25
  :vcf :chr1_33133968_C .
:chr1_33133968_C_CONTROLP26
  :dp   2 ;
  :vcf :chr1_33133968_C .
:chr1_60466814_C
  :query_id "chr1_60466814_C" ;
  :chr "1" ;
  :alt "C" ;
  :pos "60466814" .
:chr1_60466814_C_CONTROLP25
  :vcf :chr1_60466814_C .
:chr1_60466814_C_CONTROLP26
  :dp   2 ;
  :vcf :chr1_60466814_C .
:chr1_60466840_T
  :query_id "chr1_60466840_T" ;
  :chr "1" ;
  :alt "T" ;
  :pos "60466840" .
:chr1_60466840_T_CONTROLP25
  :vcf :chr1_60466840_T .
:chr1_60466840_T_CONTROLP26
  :dp   2 ;
  :vcf :chr1_60466840_T .
:chr1_62516683_C
  :query_id "chr1_62516683_C" ;
  :chr "1" ;
  :alt "C" ;
  :pos "62516683" .
:c

38865

In [15]:
#testing with rapper

!rapper -i turtle mytest.rdf

rapper: Parsing URI file:///home/jupyter/jupyter/Jane/Day%203/mytest.rdf with parser turtle
rapper: Serializing with serializer ntriples
<http://biobeat.org/rdf/ns#chr1_22848972_C> <http://biobeat.org/rdf/ns#query_id> "chr1_22848972_C" .
<http://biobeat.org/rdf/ns#chr1_22848972_C> <http://biobeat.org/rdf/ns#chr> "1" .
<http://biobeat.org/rdf/ns#chr1_22848972_C> <http://biobeat.org/rdf/ns#alt> "C" .
<http://biobeat.org/rdf/ns#chr1_22848972_C> <http://biobeat.org/rdf/ns#pos> "22848972" .
<http://biobeat.org/rdf/ns#chr1_22848972_C_CONTROLP25> <http://biobeat.org/rdf/ns#dp> "2"^^<http://www.w3.org/2001/XMLSchema#integer> .
<http://biobeat.org/rdf/ns#chr1_22848972_C_CONTROLP25> <http://biobeat.org/rdf/ns#vcf> <http://biobeat.org/rdf/ns#chr1_22848972_C> .
<http://biobeat.org/rdf/ns#chr1_22848972_C_CONTROLP26> <http://biobeat.org/rdf/ns#vcf> <http://biobeat.org/rdf/ns#chr1_22848972_C> .
<http://biobeat.org/rdf/ns#chr1_33133968_C> <http://biobeat.org/rdf/ns#query_id> "chr1_33133968_C" .
<http:

In [17]:
#upload data to 4store

import requests

myname="jane" 
rdf="mytest.rdf"
uri="http://localhost:8000/data/http://biobeat.org/"+myname+"/"+rdf
headers = {'content-type': 'application/x-turtle'}

with open(rdf, 'r') as f:
    r = requests.put(uri, data=f, headers=headers)
print(uri)
print(r) # should give a 201

http://localhost:8000/data/http://biobeat.org/jane/mytest.rdf
<Response [201]>


In [24]:
#First SPARQL query
import requests

end_point = 'http://localhost:8000/sparql/'

query = """
SELECT ?id
WHERE
{
  ?id   <http://biobeat.org/rdf/ns#chr>    "X".
}
"""

r = requests.post(end_point, data={"query": query, "output": "text"})
print(r.text)

# !cat sparql1.rq |sparql-query "http://localhost:8000/sparql/" -p

?id
<http://biobeat.org/rdf/ns#chrX_107911706_C>
<http://biobeat.org/rdf/ns#chrX_55172537_A>
<http://biobeat.org/rdf/ns#chrX_134713855_A>



In [25]:
#test in python

import requests

host = 'http://localhost:8000/sparql/'

query = """
SELECT ?s ?p ?o WHERE {
    ?s ?p ?o .
} LIMIT 10
"""

r = requests.post(host, data={ "query": query, "output": "text" })
# print(r.url)

print(r.text)

?s	?p	?o
<http://mudibo.me/rdf/ns#chr16_14340388_A>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://mudibo.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#chr11_130064131_A>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://mudibo.me/rdf/ns#snp>
<http://peter.me/rdf/ns#chr20_29628431_T>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://peter.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#chr12_30823971_G>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://mudibo.me/rdf/ns#snp>
<http://winfred/rdf/ns#chr16_88712511_T>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://winfred/rdf/ns#snp>
<http://peter.me/rdf/ns#chr1_60466814_C>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://peter.me/rdf/ns#snp>
<http://jane.me/rdf/ns#chr1_22848972_C>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://jane.me/rdf/ns#snp>
<http://jane.me/rdf/ns#chr1_22848972_C>	<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>	<http://jane.me/rdf/ns#snp>
<http://mudibo.me/rdf/ns#